In [1]:
import pandas as pd

In [2]:
df_datasets_ML = pd.read_pickle("savedDF_datasets_ML.pkl")

# MACHINE LEARNING

## Classification

In [5]:
X = df_datasets_ML[['restaurant_scale','Taxi_scale','mobike_scale']]
Y = df_datasets_ML['happiness']

In [6]:
import numpy as np

In [7]:
Z = pd.DataFrame.copy(Y)
Z[Y <= np.median(Y)] = "Unhappy"
Z[Y > np.median(Y)] = "Happy"

## Training and Testing 

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Z_train, Z_test = train_test_split(X, Z, test_size=0.3, shuffle=True)

## Decision Tree

In [9]:
# We want to determine the best CV
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
param_grid = {'max_depth': [1,2,4,6,8,10,20,40,100]}
search = GridSearchCV(tree.DecisionTreeClassifier(), param_grid, cv= ShuffleSplit(n_splits=5))
search.fit(X,Z)

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=None, test_size=None, train_size=None),
             error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [1, 2, 4, 

In [10]:
search.best_params_

{'max_depth': 2}

##### search.best_params_

In [11]:
from sklearn.model_selection import cross_val_score
clf = tree.DecisionTreeClassifier(max_depth=search.best_params_['max_depth'])

In [12]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=search.best_params_['max_depth'])
clf = clf.fit(X_train, Z_train)
prediction = clf.predict(X_test)
np.mean(prediction == Z_test)

0.775

In [13]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
tree.plot_tree(clf)

[Text(581.25, 641.6666666666666, 'X[0] <= -0.085\nentropy = 0.497\nsamples = 186\nvalue = [86, 100]'),
 Text(290.625, 385.0, 'X[1] <= -0.474\nentropy = 0.354\nsamples = 109\nvalue = [25, 84]'),
 Text(145.3125, 128.33333333333326, 'entropy = 0.221\nsamples = 79\nvalue = [10, 69]'),
 Text(435.9375, 128.33333333333326, 'entropy = 0.5\nsamples = 30\nvalue = [15, 15]'),
 Text(871.875, 385.0, 'X[0] <= 0.708\nentropy = 0.329\nsamples = 77\nvalue = [61, 16]'),
 Text(726.5625, 128.33333333333326, 'entropy = 0.231\nsamples = 45\nvalue = [39, 6]'),
 Text(1017.1875, 128.33333333333326, 'entropy = 0.43\nsamples = 32\nvalue = [22, 10]')]

## Random Forrest

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
param_grid={'n_estimators':[10,40,80,90]}
clf=RandomForestClassifier()
search=GridSearchCV(clf,param_grid,cv=5)
search.fit(X_train,Z_train)

C:\Users\sebr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [15]:
search.best_params_

{'n_estimators': 10}

In [16]:
clf=RandomForestClassifier(n_estimators=40)
clf.fit(X_train,Z_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [17]:
Z_pred=clf.predict(X_test)

In [18]:
np.mean(Z_pred == Z_test)

0.725

In [19]:
from sklearn.metrics import classification_report
pd.crosstab(Z_test, Z_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,Happy,Unhappy,All
True,,,
Happy,30,10,40
Unhappy,12,28,40
All,42,38,80


## AdaBoost

In [20]:
# We  want to determine the best param
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
param_grid = {'n_estimators': [10, 50, 100]}
clf = AdaBoostClassifier()
search = GridSearchCV(clf, param_grid, cv= 5)
search.fit(X_train, Z_train)

C:\Users\sebr\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                          base_estimator=None,
                                          learning_rate=1.0, n_estimators=50,
                                          random_state=None),
             iid='warn', n_jobs=None,
             param_grid={'n_estimators': [10, 50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [21]:
#best model
clf = AdaBoostClassifier(n_estimators=50)
clf = clf.fit(X_train, Z_train)
prediction = clf.predict(X_test)
np.mean(prediction == Z_test)

0.7125

## Results

In [23]:
df_ML_results = pd.read_excel ("results.xlsx")
df_ML_results.head()

,Solution,clean,noise,smell,Decision Tree,Random Forrest,Adaboost
0,restaurant,15,1,2,0.7250,0.6625,0.7125
1,Bicycle_part,15,1,2,0.5875,0.5250,0.5500
2,bus,15,1,2,0.6875,0.6875,0.6875
3,railway_station,15,1,2,0.6250,0.6250,0.6125
4,convenience store,15,1,2,0.7250,0.6750,0.6750


In [24]:
df_ML_results.to_pickle("savedDF_results_ML.pkl")

# Discussion

Firstly, the final goal of my model will be to give recommandation to the City of Shanghai in order to improve the wellbeing of people. So, as I said before I took into consideration several parameters that we had to estimate the wellbeing such as the number of restaurant, bus, taxi or again green spaces. 

The first step of the project was to find a way to quantify this number of POI at Shanghai. This is the reason why I decided to use the dataset of the city of Shanghai that gave us all the neighborhoods of the city. This dataset countained all the geometry of each district and at the meantime the other datasets of parameters were using not geometry object but x and y location. I chose to convert these locations into a point geometry that I'll be able to use with the geometry set of the neighborhoods. Then, I count the number of point inside each polygon and now my datasets are ready for the Machine Learning part.

For the machine learning part, I decided to use three different types of models: Decision Tree, Random Forrest and Adaboost. Each models will be as Classification because I don't want to predict numeric but here I want to predict different types of happiness (unhappy and happy only 2 because I have a small amount of data). 

The reason why I chose these 3 kind of models is that firstly, I want to have a model that is quick and explainable; it means that I prefer to have the result fast and that is easy to explain and understand. The Decision Tree is suitable for that. On top of that I wanted to make a comparison between this simpler model with 2 others models that is now not focus on a fast calculation but focus on the accuracy. I chose to study the Random Forrest and Adaboost for that. Moreover, each models will be solve using the cross validation tool that allow us better results.

In the next part, I'll try to predict the wellbeing of shanghainese using different inputs : restaurant, Bicycle park, Bus, Railway entrance and exit, Convenience store, Scenic spot, Sport, Taxi, Green spaces, Mobike.

---------------------------------------------------------------------------------------------------------------

In this part we will focus our discussion on the result provided inside the pickle above. First of all, during this part I just take into account one parameter each time. Then we will saw the results for multiple parameters at the same time. 
So, I wanted to highlight what is the solution of improving the wellbeing at Shanghai. That's why I decided to test my model with different coefficient on clean, smell and noise. For example, if I put a coefficient equal to 15 on clean then smell and noise have a coefficient of 1 it means that the people will be more sensitive of the clean parameter and we will be able to see which parameter can influence the most on the clean. Then if the City of Shanghai has to work on the clean they will be able to know on which point they have to focus their work.

#### 1) Clean 

Let's begin with the clean so we put a coefficient higher on this. Then we can observe that the maximum f1 score obtain is 73.75% with taxi's parameter. If we take the 10% higher values we can see that the restaurant, convenience store, taxi and mobike datas have the best f1 score. We can conclude that if people give more importance to the clean of a city then the city of Shanghai will have to focus their mind on these parameters. For example, we can imagine that if we have many restaurant or taxi the neighborhood will become more dirty because of the restaurant's trashes or the pollution of the taxi so the people won't be happy. 

#### 2) Noise 

We use the same principle for the noise so we put the highest coefficient on it. We observe that the 10% higher values are from restaurant, bus, sport and mobike. So it means that the city of Shanghai has to for example reduce the number of bus in order to improve the satisfaction of the noise for people or again for instance add more mobikes because bikes are not noisy. 

#### 3) Smell 

For the smell we can see that the highest value are from restaurant and massively from mobikes. So the best solution to improve this parameter of smell will be to add more mobikes inside Shanghai in order to have less smell and people will be happier. It's easy to understand because bus and taxi bring bad smell that bikes don't.

#### 4) Same coefficient 

If now we put the same coefficient on the three parameters clean, noise and smell we can see that the main thing that will give better results will be green spaces and sports. So if somewhere, all the parameter have a bad score it could be interesting for the city of Shanghai to add more green spaces and more place to do sports to improve the happiness.

#### 5) All

Then I decided to do calculations on the other combination: for example put the highest coefficient on clean and noise and the lowest coefficient on smell. I did 3 different combinations and I can now conclude on the results. 
The 10% highest score are obtained by restaurant, mobike and taxi. We can then conclude that the parameters that can influence the most on clean, noise and smell are these 3 parameters. 

#### 6) Combinaison of parameters

The last part that I want to deal with is the combinaison of multiples parameters at the same time for the machine learning. 
As we saw before, some of the parameters allow us to obtain a better f1 score depending the coefficient. We will now try to combine them in order to see if we will be able to have an even better f1 score. As we can see on the results file we tried some combinaison but we can observe that we don't have a real improvement of the f1 score. However we can clearly see that the best combinaison among these is to take into account the restaurant and mobike. Thanks to it, we can have a f1 score of 81.25% with a decision tree model. That's why, we can now conclude that if the city of Shanghai want to improve the wellbeing of shanghainese the main parameters to focus on are the restaurant and mobike ! 